In [1]:
import os
import sys
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1,2"
sys.path.append('/root/code')

import h5py as h5
from definitions import LOG_DIR, WEIGHT_DIR, DATASET_DIR
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
import time
import datetime
import numpy as np
from utils.contrastiveLoss import ContrastiveLoss
from models.Complex_SCNN18_Sigmoid import Complex_SCNN18_Sigmoid
from models.Complex_SCNN18 import Complex_SCNN18
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.callbacks import Callback
import utils.dataset as dataset
import logging
from logging import handlers

In [2]:
LOG = logging.getLogger('root')

def initLog(debug=False):
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s %(levelname)s %(message).150s',
        datefmt='%Y-%m-%d %H:%M',
        handlers=[logging.StreamHandler(), handlers.RotatingFileHandler('Complex_SCNN18.log', "w", 1024 * 1024 * 100, 3, "utf-8")]
    )
    LOG.setLevel(logging.DEBUG if debug else logging.INFO)
    tf.get_logger().setLevel('ERROR')

In [3]:
initLog()

In [4]:
class TrainingStepLogger(Callback):
    def on_epoch_end(self, epoch, logs=None):
        LOG.info(f'End of Epoch {epoch + 1} - Loss: {logs["loss"]:.4f}')

In [5]:
def get_optimizer(optimizer, lr):
    optimizer = optimizer.lower()
    if optimizer == 'adadelta':
        return tf.optimizers.Adadelta() if lr == 0 else tf.optimizers.Adadelta(learning_rate=lr)
    elif optimizer == 'adagrad':
        return tf.optimizers.Adagrad() if lr == 0 else tf.optimizers.Adagrad(learning_rate=lr)
    elif optimizer == 'adam':
        return tf.optimizers.Adam() if lr == 0 else tf.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'adamax':
        return tf.optimizers.Adamax() if lr == 0 else tf.optimizers.Adamax(learning_rate=lr)
    elif optimizer == 'sgd':
        return tf.optimizers.SGD() if lr == 0 else tf.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop':
        return tf.optimizers.RMSprop() if lr == 0 else tf.optimizers.RMSprop(learning_rate=lr)

In [6]:
# Define hyper-parameter
lr = 0.1
max_epochs = 1600
batch_size = 100
input_height = 32000
input_width = 1
nb_classes = 1

In [7]:
train_ds = dataset.load('./SCNN18_0.1second/SCNNR-Jamendo-twoSecond-oneOutput-train.h5')
dataset_length = [i for i, _ in enumerate(train_ds)][-1] + 1
train_ds = train_ds.take(int(dataset_length / 10))

# train_data = dataset.get_dataset_without_label('./SCNN18_0.1second/SCNNR-Jamendo-twoSecond-oneOutput-train.h5')
# train_label = dataset.get_ground_truth('./SCNN18_0.1second/SCNNR-Jamendo-twoSecond-oneOutput-train.h5')

dataset_length = [i for i, _ in enumerate(train_ds)][-1] + 1
print(dataset_length)

# val_ds = dataset.load('./SCNN18_0.1second/SCNNR-Jamendo-twoSecond-oneOutput-test.h5')

# val_data = dataset.get_dataset_without_label('./SCNN18_0.1second/SCNNR-Jamendo-test.h5')
# val_label = dataset.get_ground_truth('./SCNN18_0.1second/SCNNR-Jamendo-test.h5')
# val_ds = tf.data.Dataset.from_tensor_slices((val_data, val_label[:, 0]))

# dataset_length = [i for i, _ in enumerate(val_ds)][-1] + 1
# print(dataset_length)

train_ds = train_ds.batch(batch_size)
# train_ds = train_ds.batch(batch_size).shuffle(
#     dataset_length, reshuffle_each_iteration=False)
# val_ds = val_ds.batch(batch_size)

7044


In [8]:
input_shape = (input_height, input_width)

strategy = tf.distribute.MirroredStrategy(devices=[f'/gpu:{i}' for i in range(3)])
with strategy.scope():
    model = Complex_SCNN18_Sigmoid(input_shape, nb_classes).model()

    optimizer = get_optimizer('adam', lr)

    # model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    model.summary(line_length=250, print_fn=LOG.info)

training_step_logger = TrainingStepLogger()
model.fit(train_ds,
          epochs=max_epochs,
          callbacks=[training_step_logger])

model.save_weights(os.path.join(WEIGHT_DIR, f"{str(datetime.date.today())}_Complex_SCNN18_Sigmoid_twoSecond_oneOutput_square_training_0.1LR.h5"))

2024-03-11 16:09 INFO Model: "functional_1"
2024-03-11 16:09 INFO ______________________________________________________________________________________________________________________________________________________
2024-03-11 16:09 INFO Layer (type)                                                                      Output Shape                                           Param #      
2024-03-11 16:09 INFO ======================================================================================================================================================
2024-03-11 16:09 INFO input (InputLayer)                                                                [(None, 32000, 1)]                                     0            
2024-03-11 16:09 INFO ______________________________________________________________________________________________________________________________________________________
2024-03-11 16:09 INFO stft_network (STFT_network)                                          

Epoch 1/1600
71/71 [==============================] - ETA: 0s - loss: 8.4984 - accuracy: 0.5070

2024-03-11 16:10 INFO End of Epoch 1 - Loss: 8.4984


71/71 [==============================] - 20s 287ms/step - loss: 8.4984 - accuracy: 0.5070
Epoch 2/1600
71/71 [==============================] - ETA: 0s - loss: 2.7854 - accuracy: 0.5772

2024-03-11 16:10 INFO End of Epoch 2 - Loss: 2.7854


...

71/71 [==============================] - 21s 290ms/step - loss: 8.7491 - accuracy: 0.6288
Epoch 1599/1600
71/71 [==============================] - ETA: 0s - loss: 5.9557 - accuracy: 0.6455

2024-03-12 03:59 INFO End of Epoch 1599 - Loss: 5.9557


71/71 [==============================] - 20s 285ms/step - loss: 5.9557 - accuracy: 0.6455
Epoch 1600/1600
71/71 [==============================] - ETA: 0s - loss: 3.9879 - accuracy: 0.6685

2024-03-12 03:59 INFO End of Epoch 1600 - Loss: 3.9879


71/71 [==============================] - 21s 289ms/step - loss: 3.9879 - accuracy: 0.6685
